In [7]:
import numpy as np
import pandas as pd

# df = pd.read_csv("FrankensteinFullForm.csv")
# df = df.as_matrix()
# print(df.shape)
# labels = pd.read_csv("LabelFrankensteinFullForm.csv", header = None)
# labels = labels.as_matrix()

<h1>Load Dataset</h1>

In [75]:
import pickle
from numpy import genfromtxt
from sklearn.utils import shuffle
import sklearn

TRAIN_RATIO = 0.8
AMOUNT_OF_CLASSES = 7

definition = ['static', 'move', 'rot', 'frot', 'light', 'dynamiclight','unknown']

def normalize(dataset):
    datasetTmp = np.copy(dataset)
    maxx = np.max(dataset, axis=0)
    minn = np.min(dataset, axis=0)
    rangee = maxx - minn
    datasetTmp -= minn
    datasetTmp /= rangee
    return datasetTmp

def loadDataset(path):
    dataset = (genfromtxt(path, delimiter=','))[1:,:]
    dataset = normalize(dataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    training_dataset = dataset[0:trainingAmount,:]
    test_dataset = dataset[trainingAmount:,:]
    print("data: ", dataset.shape)
    return training_dataset.astype(float), test_dataset.astype(float)

def loadLabel(data_path, labels_path):
    dataset = (genfromtxt(data_path, delimiter=','))[1:,:]
    dataset = normalize(dataset)
    trainingAmount = int(TRAIN_RATIO * dataset.shape[0])
    #     labels = pickle.load(INIT_PATH + path)
    with open(labels_path, 'rb') as handle:
        labels = pickle.load(handle)
    ###### TEMPORARY
    del labels[-1]
    ######
    all_labels = np.zeros((len(labels), AMOUNT_OF_CLASSES))
#     training_labels = np.zeros((trainingAmount, AMOUNT_OF_CLASSES))
#     test_labels = np.zeros((len(labels) - trainingAmount, AMOUNT_OF_CLASSES))
    label_counter = 0
    per_label = [0] * AMOUNT_OF_CLASSES
#     print(labels)
    for label in labels:
        tmp = np.zeros((1,AMOUNT_OF_CLASSES))
        if (label == "static"):
            tmp[0,0] = 1
            per_label[0] += 1
        elif (label == "move"):
            tmp[0,1] = 1
            per_label[1] += 1
        elif (label == "rot"):
            tmp[0,2] = 1
            per_label[2] += 1            
        elif (label == "frot"):
            tmp[0,3] = 1
            per_label[3] += 1
        elif (label == "light"):
            tmp[0,4] = 1
            per_label[4] += 1
        elif (label == "dynamiclight"):
            tmp[0,5] = 1
            per_label[5] += 1
        else:
            tmp[0,6] = 1
            per_label[6] += 1
        all_labels[label_counter,:] = tmp
#         if(label_counter < trainingAmount):
#             training_labels[label_counter,:] = tmp
#         else:
#             test_labels[label_counter - trainingAmount,:] = tmp
        label_counter += 1
    dataset, all_labels = shuffle(dataset, all_labels, random_state=0)
    return assignData(dataset, all_labels)
    
#     train_dataset = dataset[0:trainingAmount,:]
#     test_dataset = dataset[trainingAmount:,:]
#     print("data: ", dataset.shape)
    
#     train_labels = all_labels[:trainingAmount,:]
#     test_labels = all_labels[trainingAmount:,:]
    
#     print(", ".join([definition[i] + ": " + str(per_label[i]) for i in range (len(per_label))]))
#     return train_dataset.astype(float), test_dataset.astype(float),train_labels, test_labels

In [81]:
def assignData(data,labels):
    UNKNOWN_AMOUNT = 5
    train_amount = int(data.shape[0] * TRAIN_RATIO)
    AMOUNT_PER_CLASS = int((data.shape[0] - UNKNOWN_AMOUNT) * TRAIN_RATIO)
    class_count = [0] * train_amount
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    for i in range (data.shape[0]):
        if(len(train_data) == train_amount):
            test_data.append(data[i])
            test_labels.append(labels[i])
        else:
            index = np.argmax(data[i])
            if(index == AMOUNT_OF_CLASSES - 1):
                test_data.append(data[i])
                test_labels.append(labels[i])
            if( class_count[index] < AMOUNT_PER_CLASS) :
                train_data.append(data[i])
                train_labels.append(labels[i])
                class_count[index] += 1
            else:
                test_data.append(data[i])
                test_labels.append(abels[i])
    return np.asarray(train_data), np.asarray(test_data), np.asarray(train_labels), np.asarray(test_labels)
                    

In [83]:
train_data, test_data, train_labels, test_labels = loadLabel("FrankensteinFullForm.csv","LabelFrankensteinFullForm.p")
print(train_labels.dtype)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)
# print(train_labels)
print(np.sum(train_labels,axis=0))

float64
(110, 10)
(33, 10)
(110, 7)
(33, 7)
[19. 16. 17. 17. 18. 18.  5.]


In [84]:
with open("FrankensteinFullForm.csv") as f:
    lines = (line for line in f if not line.startswith('#'))
    dataset = np.loadtxt(lines, delimiter=',', skiprows=1)

In [85]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras import optimizers

In [86]:
model = Sequential()

model.add(Dense(7, activation='linear', input_dim=10))

In [90]:
sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

In [91]:
model.fit(train_data,train_labels,batch_size=1, epochs=100, verbose=1)

Epoch 1/100
110/110 [==============================] - 0s 2ms/step - loss: 0.0884 - acc: 0.6818
Epoch 2/100
110/110 [==============================] - 0s 900us/step - loss: 0.0884 - acc: 0.6818
Epoch 3/100
110/110 [==============================] - 0s 882us/step - loss: 0.0884 - acc: 0.6818
Epoch 4/100
110/110 [==============================] - 0s 829us/step - loss: 0.0884 - acc: 0.6818
Epoch 5/100
110/110 [==============================] - 0s 902us/step - loss: 0.0884 - acc: 0.6727
Epoch 6/100
110/110 [==============================] - 0s 938us/step - loss: 0.0884 - acc: 0.6818
Epoch 7/100
110/110 [==============================] - 0s 904us/step - loss: 0.0884 - acc: 0.6818
Epoch 8/100
110/110 [==============================] - 0s 836us/step - loss: 0.0884 - acc: 0.6818
Epoch 9/100
110/110 [==============================] - 0s 803us/step - loss: 0.0884 - acc: 0.6818
Epoch 10/100
110/110 [==============================] - 0s 822us/step - loss: 0.0883 - acc: 0.6818
Epoch 11/100
110/110 

In [92]:
score = model.evaluate(test_data, test_labels, verbose=0)
print(score)

[0.09270871995073376, 0.6363636363636364]
